# Importante 

* Tener en cuenta bajadas de Procedimientos, Personas Armas, Divisas , vehiculos (secuestrados, ministerio), Narcotrafico general,Objetos y pegar en la carpeta bajadas de fecha del primero del 1 del mes en informar hasta fecha del informe 

### Importacion de librerias
importacion de libreias para tranformacion de datos

In [1]:
import pandas as pd
import re
import numpy as np
from openpyxl import load_workbook

### Parametros
Coloco los parametros a tener en cuenta para donde se encuentra los archivos de bajada

In [2]:
LUGARES_CATALOGADOS= {
                    "SECTOR RESTRINGIDO":	"SECTOR DE SEGURIDAD RESTRINGIDA AEROPORTUARIA",
                    "SECTOR PUBLICO"	:"INSTALACIONES DE ACCESO AL PUBLICO GENERAL",
                    "PERIMETRO DE JURISDICCION":	"PERIMETRO AEROPORTUARIO",
                    "FUERA DE JURISDICCION":	"DESPLIEGUE",
                    "TRANSPORTE PUBLICO": "TRANSPORTE PUBLICO",
                    "S/D": "S/D"
}

UNIDADES_MUNICIPIOS = {
    "EZE": "JOSÉ M. EZEIZA",
    "AER": "COMUNA 14",
    "SFO": "SAN FERNANDO",
    "BHI": "BAHÍA BLANCA",
    "MDP": "GENERAL PUEYRREDÓN",
    "MDZ": "LAS HERAS",
    "COR": "CAPITAL",
    "JUA": "9 DE JULIO",
    "LUI": "JUAN MARTÍN DE PUEYRREDÓN",
    "MLG": "MALARGÜE",
    "RAF": "SAN RAFAEL",
    "RCU": "RÍO CUARTO",
    "SRO": "CAPITAL",
    "JUJ": "EL CARMEN",
    "SAL": "LA CAPITAL",
    "CAT": "VALLE VIEJO",
    "LAR": "CAPITAL",
    "SGO": "CAPITAL",
    "TRH": "RÍO HONDO",
    "TUC": "CRUZ ALTA",
    "IGU": "IGUAZÚ",
    "FSA": "FORMOSA",
    "ROS": "ROSARIO",
    "RCQ": "GENERAL OBLIGADO",
    "POS": "CAPITAL",
    "CRR": "CAPITAL",
    "LIB": "PASO DE LOS LIBRES",
    "PAR": "PARANÁ",
    "RES": "SAN FERNANDO",
    "SVO": "LA CAPITAL",
    "BAR": "BARILOCHE",
    "TRE": "RAWSON",
    "NEU": "CONFLUENCIA",
    "CAL": "LAGO ARGENTINO",
    "CHP": "LÁCAR",
    "CRV": "ESCALANTE",
    "PMY": "BIEDMA",
    "ESQ": "FUTALEUFÚ",
    "GAL": "GÜER AIKE",
    "GDE": "RÍO GRANDE",
    "USU": "USHUAIA",
    "VIE": "ADOLFO ALSINA",
    "RG4": "-",
}

PROVINCIAS = {
    "C.A. BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "BUENOS AIRES": "BUENOS AIRES",
    "SANTA CRUZ": "SANTA CRUZ",
    "SANTA FE": "SANTA FE",
    "MENDOZA": "MENDOZA",
    "TUCUMÁN": "TUCUMAN",
    "TIERRA DEL FUEGO": "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLANTICO SUR",
    "SALTA": "SALTA",
    "NEUQUÉN": "NEUQUEN",
    "CÓRDOBA": "CORDOBA",
    "LA PAMPA": "LA PAMPA",
    "RÍO NEGRO": "RIO NEGRO",
    "CATAMARCA": "CATAMARCA",
    "MISIONES": "MISIONES",
    "SAN JUAN": "SAN JUAN",
    "CORRIENTES": "CORRIENTES",
    "CHUBUT": "CHUBUT",
    "FORMOSA": "FORMOSA",
    "JUJUY": "JUJUY",
    "SANTIAGO DEL ESTERO": "SANTIAGO DEL ESTERO",
    "LA RIOJA": "LA RIOJA",
    "CHACO": "CHACO",
    "ENTRE RÍOS": "ENTRE RIOS",
    "SAN LUIS": "SAN LUIS"
}


UNIDADES= {
        "DROPA I": "UR1"
}

PERSONAL = {    
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}

SCANNER = {
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}


# Procedimientos
### Funciones 
declaraciones de funciones procesar cada celda y columna del informe del sigipol como de operaciones

In [3]:

contador_global_sigipol = {}
def generar_codigo_sigipol(row):
    id_operativo = row['ID_OPERATIVO']
    if id_operativo not in contador_global_sigipol:
        contador_global_sigipol[id_operativo] = 0
    contador_global_sigipol[id_operativo] += 1
    contador = contador_global_sigipol[id_operativo]
    return id_operativo + "-(" + str(contador) + ")"
        
def procesar_descripcion(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if tipo == "DENUNCIA":
        return "DENUNCIA POLICIAL"
    elif tipo == "CONTROL PREVENTIVO":
        return f"CONTROL PREVENTIVO - {procesar_lugar(row)}"
    elif tipo == "ORDEN DE ALLANAMIENTO":
        return "ORDEN DE ALLANAMIENTO"
    elif tipo == "ORDEN DE ALLANAMIENTO / DETENCIÓN":
        return "ORDEN DE ALLANAMIENTO"
    else:
        return "OTRO MANDATO JUDICIAL"

def procesar_tipo(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if pd.isna(tipo):
        return ""
    elif tipo == "DENUNCIA"  or tipo == "CONTROL PREVENTIVO" :
        return "ORDEN POLICIAL"
    else:
        return "ORDEN JUDICIAL"
    
def procesar_provincia(row):
    provincia = row['PROVINCIA']
    if pd.isna(provincia):
        return ""
    return PROVINCIAS.get(provincia, provincia)
    
def procesar_municipio(row):
    unidad = row['UOSP']
    if pd.isna(unidad):
        return ""
    return UNIDADES_MUNICIPIOS.get(unidad, unidad)

def procesar_lugar(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    return  LUGARES_CATALOGADOS[lugar]

def procesar_direccion(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    ciudad = row['CIUDAD']
    if lugar == "FUERA DE JURISDICCION" and ciudad == "ROSARIO":
        return "-"
    elif lugar == "FUERA DE JURISDICCION":
        return str(row['CALLE']) + " " + str(row['NUMERO']) + ", " + str(row['CIUDAD']) + " - " + str(row['PARTIDO'])
    else:
        return "-"
    
def controlar_estado (row):
    ursa = row['URSA'] 
    unidad = row['UOSP'] 
    estado = row['ESTADO_PARTE'] 
    if pd.isna(unidad)  and ursa == 'RG4' and estado == 'NO DISPONIBLE ESTADISTICA':
        return  "DISPONIBLE ESTADISTICA"
    else:
        return estado 
    
def leer_excel_a_df(worksheet):
    data = []
    titulos = [worksheet.cell(row=3, column=col).value for col in range(2, worksheet.max_column + 1)]
    for row in worksheet.iter_rows(min_row=4, min_col=2, max_col=worksheet.max_column, values_only=True):
        data.append(row)
    
    df = pd.DataFrame(data, columns=titulos)
    return df



# Base de Datos
### Leer base de datos

leer la base de datos "data/Base_informada" 


In [4]:
df_base_informada = load_workbook("data/BASE_PROCESADO.xlsx")
hoja = df_base_informada['GEOG. PROCEDIMIENTO']

# Crear un DataFrame a partir de los datos
df_base_informada = leer_excel_a_df(hoja)


# Procedimientos
### Ingresar bajada de procedimiento

* convierto las bajadas en el dataframe para poder manipularlos y poder hacer calculos

In [5]:
excel_bajada_procedimientos = pd.read_excel('bajadas/bajada_general.xls')


### Filtrar procedimientos 
solo que esta que estan disponible en estadistica y las fechas estipuladas de información 

In [6]:
excel_bajada_procedimientos['ESTADO_PARTE'] = excel_bajada_procedimientos.apply(controlar_estado ,axis=1).copy()
excel_bajada_procedimientos['UOSP'] = excel_bajada_procedimientos['UOSP'].fillna(excel_bajada_procedimientos['URSA'])
excel_bajada_procedimientos['GEOREFERENCIA_X'] = excel_bajada_procedimientos['GEOREFERENCIA_X'].fillna('-')
excel_bajada_procedimientos['GEOREFERENCIA_Y'] = excel_bajada_procedimientos['GEOREFERENCIA_Y'].fillna('-')

excel_bajada_procedimientos = excel_bajada_procedimientos[
    (excel_bajada_procedimientos['ESTADO_PARTE'] != 'NO DISPONIBLE ESTADISTICA') 
].copy()


In [7]:
def procesar_causa_judicial(row):
    causa = row['CAUSAJUDICIALNUMERO']
    tipo = row['TIPO_CAUSA_INTERNA']
    causa_int = row['CAUSA_INTERNA_NUMERO'].strip()
    # Convertir la causa a string y eliminar espacios al inicio y final
    causa_str = str(causa).strip()
    print (causa, causa_int)
    # Lista de prefijos que se quieren eliminar (sin importar mayúsculas o minúsculas)
    prefijos = ["NRO",  "N°", "EXPTE", "EXPEDIENTE", "EXPT",  "." ,"-N"]
    

    for prefijo in prefijos:
        causa_str = causa_str.replace(prefijo, "")
    # Lista de tipos especiales que se tratan como "tipo" en lugar de ser parte de la causa
    
    tipos_especiales = ["CASO", "CARPETA", "CAUSA" ,"FPO", "FSM","FLP", "MPF", "IPP","FMZ","CCC","CPE" ,"INS", "GAR"]
    
    # Verificar si la causa comienza con algún tipo especial
    for tipo_especial in tipos_especiales:
        if causa_str.upper().startswith(tipo_especial):
            # Extraer el tipo especial y quitarlo del inicio de la causa
            prefijo = tipo_especial 
            resto_causa = causa_str[len(tipo_especial):].strip()
            print(f"Prefijo: {prefijo}")
            print(f"Resto de la causa: {resto_causa}")
            
            causa_str = prefijo  +"-"+ resto_causa
            
            

    
    conjunto = ""
    if tipo == "ACTUACIÓN JUDICIAL" or tipo == "RESTRICCIÓN A LA LIBERTAD":
        conjunto = tipo + "-" + causa_int
    else:
        conjunto = ( tipo + "-" + causa_int ).replace(" ", "-")
    conjunto = conjunto.replace("--","-")
    # Si la causa original está vacía o contiene ciertos valores, devolver el conjunto
    if pd.isna(causa) or causa in ["", "S/D", "A/S", "N/C"]:
        return conjunto
    
    return causa_str


### Filtros de procedimientos
Ejecucion de funciones para la bajada de procedimientos de sigipol  tranformando los datos para que se muestren como lo requiere la DNEC

In [8]:
df_procedimientos = pd.DataFrame()

df_procedimientos['ID_OPERATIVO'] = excel_bajada_procedimientos.apply(procesar_causa_judicial, axis=1)
df_procedimientos['ID_PROCEDIMIENTO'] = df_procedimientos.apply(generar_codigo_sigipol, axis=1)
df_procedimientos['UNIDAD_INTERVINIENTE'] = excel_bajada_procedimientos['UOSP']
df_procedimientos['DESCRIPCIÓN'] = excel_bajada_procedimientos.apply(procesar_descripcion, axis=1)
df_procedimientos['TIPO_INTERVENCION'] = excel_bajada_procedimientos.apply(procesar_tipo, axis=1)
df_procedimientos['PROVINCIA'] = excel_bajada_procedimientos.apply(procesar_provincia, axis=1)
df_procedimientos['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos.apply(procesar_municipio, axis=1)
df_procedimientos['LOCALIDAD'] = "-"
df_procedimientos['DIRECCION'] = excel_bajada_procedimientos.apply(procesar_direccion, axis=1)
df_procedimientos['LATITUD'] = excel_bajada_procedimientos['GEOREFERENCIA_Y']
df_procedimientos['LONGITUD'] = excel_bajada_procedimientos['GEOREFERENCIA_X']
df_procedimientos['FECHA'] = pd.to_datetime(excel_bajada_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_procedimientos['HORA'] = pd.to_datetime(excel_bajada_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_procedimientos['FUERZA_INTERVINIENTE'] = "PSA"
df_procedimientos['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_procedimientos['PASO_FRONTERIZO'] = "-"
df_procedimientos['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_procedimientos['Observaciones - Detalles'] = "-"


df_procedimientos = df_procedimientos[['FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]


df_procedimientos = df_procedimientos[~df_procedimientos['ID_PROCEDIMIENTO'].isin(df_base_informada['ID_PROCEDIMIENTO'])].copy()


nan 0379-EZE/24
nan 0056-SAL/24
nan 0261-EZE/24
nan 0119-RG4/24
nan 0139-BAR/24
nan 0126-COR/24
nan 0139-NEU/24
nan 0127-COR/24
nan 0008-MDP/24
nan 0140-NEU/24
nan 0128-COR/24
FPO 4797/2024 0082-IGU/24
Prefijo: FPO
Resto de la causa: 4797/2024
nan 0002-RAF/24
nan 0000-AER/24
nan 0120-RG4/24
nan 0552-AER/24
nan 0141-NEU/24
nan 0057-SAL/24
33264 0122-EZE/24
nan 0129-MDZ/24
nan 0553-AER/24
41475/2022 0103-USU/24
nan 0010-TRE/24
nan 0024-SGO/24
nan 0554-AER/24
nan 0142-NEU/24
nan 0009-CRR/24
FPO 4834/2024 0083-IGU/24
Prefijo: FPO
Resto de la causa: 4834/2024
CAUSA NRO. 9707/18 0050-AER/24
Prefijo: CAUSA
Resto de la causa: 9707/18
nan 0143-NEU/24
FPO 4837/2024 0084-IGU/24
Prefijo: FPO
Resto de la causa: 4837/2024
FPO 4838/2024 0085-IGU/24
Prefijo: FPO
Resto de la causa: 4838/2024
nan 0123-EZE/24
nan 0262-EZE/24
nan 0380-EZE/24
nan 0381-EZE/24
nan 0382-EZE/24
nan 0008-FSA/24
nan 0140-BAR/24
nan 0144-NEU/24
nan 0129-COR/24
nan 0130-COR/24
nan 0384-EZE/24
nan 0263-EZE/24
CPE 588/2024 0027-EZE/

# Operaciones
### Funciones de filtro para operaciones

In [9]:

def filas_diferentes(row, df_base_informada):
    # Buscar si hay alguna fila en df_base_informada que sea idéntica en las columnas especificadas
    match = df_base_informada[
        (df_base_informada['ID_OPERATIVO'] != row['ID_OPERATIVO']) &
        (df_base_informada['FECHA'] != row['FECHA']) &
        (df_base_informada['LATITUD'] != row['LATITUD']) &
        (df_base_informada['LONGITUD'] != row['LONGITUD'])
    ]
    # Si no se encuentra un match, significa que la fila es diferente
    return match.empty

def procesar_unidad (row):
    unidad = row['UNIDAD_INTERVINIENTE']
    return  UNIDADES.get(unidad, unidad)


def colocar_guion_espacio(texto):
    # Eliminar espacios adicionales al inicio o al final
    texto = texto.strip()
    # Remover "N°" si está presente
    texto = texto.replace("N°", "").strip()
    # Reemplazar espacios por guiones
    texto = texto.replace(" ", "-")
    # Remover puntos innecesarios
    texto = texto.replace(".", "")

    # Manejar prefijos específicos
    prefijos = ['OSR', 'OSL', 'OSC']
    
    for prefijo in prefijos:
        if texto.upper().startswith(prefijo):
            # Eliminar el prefijo y agregarlo nuevamente con un guion si no lo tiene
            texto = texto[len(prefijo):].strip()
            if not texto.startswith("-"):
                texto = prefijo + "-" + texto
            if "(" in texto and not "-(" in texto:
                texto = texto.replace("(", "-(")
            break

    print(texto)
    # Reemplazar cualquier instancia de múltiples guiones consecutivos con un solo guion
    texto = re.sub(r'-+', '-', texto)
    
    
    texto=  formatear_contador(texto)
    return texto


def formatear_contador(texto):
    # Eliminar cualquier "-(número)" o "--(número)" al final del texto
    texto_procesado = re.sub(r'-+\(\d+\)$', '', texto)

    return texto_procesado

* Filtros de Ordenes de servicios operaciones
Ejecucion de funciones para la planilla enviada por operaciones tranformando los datos para que se muestren como lo requiere la DNEC

In [10]:
excel_bajada_operaciones = load_workbook("bajadas/operaciones.xlsx")
hoja_geog = excel_bajada_operaciones['GEOG. PROCEDIMIENTO']
hoja_vehi = excel_bajada_operaciones['VEHI. Y PERSO. CONTROLADAS']


df_operaciones = leer_excel_a_df(hoja_geog)
df_operaciones = df_operaciones.dropna(subset=['ID_PROCEDIMIENTO'], how='any').copy()
df_operaciones['ID_PROCEDIMIENTO'] = df_operaciones['ID_PROCEDIMIENTO'].apply(colocar_guion_espacio)
df_operaciones['UNIDAD_INTERVINIENTE'] = df_operaciones.apply(procesar_unidad, axis=1)
df_operaciones['ID_OPERATIVO'] = df_operaciones['ID_PROCEDIMIENTO'].copy()


df_operaciones['FECHA'] = pd.to_datetime(df_operaciones['FECHA'], errors='coerce')

# Aplicar la función para identificar las filas que son diferentes
print(df_operaciones.count())
df_operaciones = df_operaciones[df_operaciones.apply(filas_diferentes, axis=1, df_base_informada=df_base_informada)].copy()
print(df_operaciones.count())

conteo_base_datos = df_base_informada['ID_OPERATIVO'].value_counts()
conteo_acumulado  = conteo_base_datos.to_dict()

for index, row in df_operaciones.iterrows():
    id_operativa = row['ID_OPERATIVO']
    
    # Verificar cuántas veces ha aparecido el ID_operativa en total hasta ahora (base + nuevos)
    if id_operativa in conteo_acumulado:
        conteo_acumulado[id_operativa] += 1
    else:
        conteo_acumulado[id_operativa] = 1
    
    # El nuevo ID_procedimiento será el ID_operativa más el número de veces que se ha repetido, dentro de paréntesis
    nuevo_id_procedimiento = f"{id_operativa}-({conteo_acumulado[id_operativa]})"
    
    # Asignar el nuevo ID_procedimiento solo en la columna ID_procedimiento
    df_operaciones.at[index, 'ID_PROCEDIMIENTO'] = nuevo_id_procedimiento

OSR-0843-UR1/2024-(10)
OSR-0834-UR4/2024-(36)
OSR-0834-UR4/2024-(37)
OSR-0834-UR4/2024-(38)
OSR-0834-UR4/2024-(39)
OSR-0852-UR4/2024-(24)
OSR-0852-UR4/2024-(25)
OSR-0852-UR4/2024-(26)
OSR-0852-UR4/2024-(27)
OSR-0442-UR5/2024-(12)
0159--TRE/2024
OSR-0843-UR1/2024-(10)
OSR-0834-UR4/2024-(40)
OSR-0852-UR4/2024-(28)
OSR-0852-UR4/2024-(29)
OSR-0852-UR4/2024-(30)
OSR-0852-UR4/2024-(31)
OSR-0442-UR5/2024-(13)
OSR-0843-UR1/2024-(11)
OSR-0834-UR4/2024-(41)
OSR-0852-UR4/2024-(32)
OSR-0852-UR4/2024-(33)
OSR-0852-UR4/2024-(34)
OSR-0852-UR4/2024-(35)
OSR-0442-UR5/2024-(14)
0163-TRE/2024
OSR-0843-UR1/2024-(12)
OSR-0843-UR1/2024-(13)
OSR-0834-UR4/2024-(42)
OSR-0834-UR4/2024-(43)
OSR-0852-UR4/2024-(36)
OSR-0852-UR4/2024-(37)
OSR-0442-UR5/2024-(15)
OSR-0843-UR1/2024-(14)
OSR-0843-UR1/2024-(15)
OSR-0834-UR4/2024-(44)
OSR-0852-UR4/2024-(38)
OSR-0852-UR4/2024-(39)
OSR-0442-UR5/2024-(16)
OSR-0843-UR1/2024-(16)
OSR-0834-UR4/2024-(45)
OSR-0852-UR4/2024-(40)
OSR-0852-UR4/2024-(41)
OSR-0442-UR5/2024-(17)
OSR-0

### Guardado de informaciópn

Ordeno la informacion como de la hoja de procedimientos segun el la plainilla modelo y guardo en la planilla informe 

In [11]:
df_geog = pd.concat([df_operaciones, df_procedimientos])

nombre_archivo_modelo = "Planilla_modelo.xlsx"
nombre_archivo_informe = "Planilla_informe.xlsx"
wb = load_workbook(nombre_archivo_modelo)
ws = wb['GEOG. PROCEDIMIENTO']

# Escribir los datos del DataFrame desde la fila 4
for row_num, row in enumerate(df_geog.itertuples(index=False), start=4):
    for col_num, value in enumerate(row, start=2):  # Comienza desde la columna B (número 2)
        ws.cell(row=row_num, column=col_num).value = value

# Guardar los cambios
wb.save(nombre_archivo_informe)

print(f"\nArchivo Excel '{nombre_archivo_informe}' modificado con éxito.")


Archivo Excel 'Planilla_informe.xlsx' modificado con éxito.


Control de salidad de la informacion

In [12]:
# Aplicar la función procesar_causa_judicial a la copia

# # Contar cuántos casos se procesaron vs. cuántos se mantuvieron igual
# procesados = (wb_nueva_hoja['CAUSAJUDICIALNUMERO'] != wb_nueva_hoja['ID_OPERATIVO']).sum()
# print(f"\nCasos procesados: {procesados}")
# print(f"Casos sin cambios: {len(wb_nueva_hoja) - procesados}")